# NoteBook for numbers of Athletes 
This is the notebook for getting the information on the number of athletes and gender split for every country at every games. Then files where given to us by the Olympic Study Center 

In [1]:
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline
from bs4 import BeautifulSoup
import webbrowser
import urllib.request
from lxml import html
import zipfile
import re
import string
import sys, os
from IPython.display import Image

# The Summer Olympics File
* Two xls files sent to us by the olympic study center contained information on the number of athletes split male and females sent to each of the Olympic games by each country. The first was the summer games. 
* Each summer games had its own xls sheet titled just by the year of that games.
* The was some useless information on each of the sheets that we had to get rid of first then we could start to extract the numbers.
* After that we had to change the layout of the xls files in order to turn it into a useable dataframe. 


* This is an example of what a sheet would look like. So what we wanted to do was join these five section together to make a df for that given year. With the end goal of then joining all the years to have a big dataFrame with the number of all athletes sent from all countrys for every summer gmaes. 

![title](..\..\data\raw\Study_center\xlsEG.PNG)

In [2]:
# Ensure the file exists
if not os.path.exists( r"..\..\data\raw\Study_Center\SummerGamesATH.xlsx" ):
    print("Missing dataset file")

In [3]:
# reading in the xls file 
xlsS = pd.ExcelFile(r"..\..\data\raw\Study_Center\SummerGamesATH.xlsx")

We are also going to add the city and country that hosted the gmaes of the given year. 

In [4]:
# Dictionary for the host countries key is the year of the games and the country is the value 
countrySDict =  {2012: 'UK', 2016: 'BRA', 1976: 'CAN', 1964: 'JPN', 1984: 'USA', 1996: 'USA', 1960: 'ITA', 2004: 'GRC', 2008: 'CHN', 1988: 'KOR', 1992: 'ESP', 2000: 
       'AUS', 1972: 'DEU', 1968: 'MEX', 1980: 'RUS' }

In [5]:
# Dictionary for the host cities key is the year of the games and the city is the value 
citySDict = { 2008: 'Beijing', 1988: 'Seoul', 1996: 'Atlanta', 2016: 'Rio', 2012: 'London', 2000: 'Sydney', 2004: 'Athens',
      1980: 'Moscow', 1984: 'Los Angeles', 1992: 'Barcelone', 1964: 'Tokyo', 1968: 'Mexico', 1972: 'Munich', 1976:
       'Montréal', 1960: 'Rome'}

In [6]:
# Emoty list to hold the dataFrames one for each summer games 
SdataFrameList = []
# Starting with the 1960 olympics 
year = 1960

# For loop of 15 iterations one for every sheet/ games in the xls file 
for x in range(15):
    df = pd.read_excel(xlsS, str(year))

    #First we'll drop all the null columns with no information 
    df = df.drop(df.columns[[4, 9, 14, 19]], axis=1)
    
    # Then we created a small df fro each of the 5 dfs of 4 coulmns seen in the image above 
    j1 = df[['NOC', 'M', 'F', 'Total']].copy()
    j2 = df[['NOC.1', 'M.1', 'F.1', 'Total.1']].copy()
    j3 = df[['NOC.2', 'M.2', 'F.2', 'Total.2']].copy()
    j4 = df[['NOC.3', 'M.3', 'F.3', 'Total.3']].copy()
    j5 = df[['NOC.4', 'M.4', 'F.4', 'Total.4']].copy()
    
    # then we created a list of the names of each column so we could change all the sub dfs column to be the same for joining 
    j2n = list(j2)
    j3n = list(j3)
    j4n = list(j4)
    j5n = list(j5)
    
    # Then we could rename the columns of the 5 smaller dfs
    j2 = j2.rename(columns={ j2n[0]: 'NOC', j2n[1]: 'M', j2n[2]: 'F', j2n[3]: 'Total'})
    j3 = j3.rename(columns={ j3n[0]: 'NOC', j3n[1]: 'M', j3n[2]: 'F', j3n[3]: 'Total'})
    j4 = j4.rename(columns={ j4n[0]: 'NOC', j4n[1]: 'M', j4n[2]: 'F', j4n[3]: 'Total'})
    j5 = j5.rename(columns={ j5n[0]: 'NOC', j5n[1]: 'M', j5n[2]: 'F', j5n[3]: 'Total'})
    
    # Here we are joining the 5 smaller dfs and making one big one
    df = pd.concat([j1, j2, j3, j4, j5], ignore_index=True)
    
    # Dropping all the null values these null rows would be the whitespace rows seen in the image above 
    df = df.dropna(axis=0, how='any')
    
    # Adding a year column with the year of the given games 
    df['Year'] = year
    
    # Adding the host country column referencing its respective dictionary  
    df['Host_Country'] = countrySDict[year]
    
   # Adding the host country column referencing its respective dictionary   
    df['Host_City'] = citySDict[year]
    
    # Creating a list of dataframes each df in the list represents the numbers from one summer games 
    SdataFrameList.append(df)
    
    # Changing the year to reference the next sheet in the xls file
    year = year + 4    

In [7]:
# Now we create the final dataFrame containing all the summer games and the infomration on athletes sent 
sumdf = pd.concat(SdataFrameList, ignore_index=True)

In [8]:
# creating the summer and winter games bolleans
sumdf['Summer']= True
sumdf['Winter']= False

In [9]:
sumdf.head(5)

,NOC,M,F,Total,Year,Host_Country,Host_City,Summer,Winter
0,EGY,74.0,0.0,74.0,1960,ITA,Rome,True,False
1,ETH,9.0,0.0,9.0,1960,ITA,Rome,True,False
2,GHA,13.0,0.0,13.0,1960,ITA,Rome,True,False
3,KEN,28.0,0.0,28.0,1960,ITA,Rome,True,False
4,LBR,4.0,0.0,4.0,1960,ITA,Rome,True,False


# The Winter Olympics File
* The winter Olympics file was the same idea. It was an xls file containing multiple sheets each one for a different winter games. 

In [10]:
# Ensure the file exists
if not os.path.exists(r"..\..\data\raw\Study_center\WinterGamesATH.xlsx" ):
    print("Missing dataset file")

In [11]:
# reading in the xls file 
xlsW = pd.ExcelFile(r"..\..\data\raw\Study_Center\WinterGamesATH.xlsx")

In [12]:
# Dictionary for the host cities key is the year of the games and the city is the value 
cityWDict = { 1972: 'Sapporo', 1968: 'Grenoble', 2002: 'Salt Lake City', 1980: 'Lake Placid', 2018: 'PyeongChang', 
1992: 'Albertville', 
1998: 'Nagano', 
2006: 'Turin', 
2014: 'Sochi', 
2010: 'Vancouver', 
1984: 'Sarajevo', 
1988: 'Calgary', 
1964: 'Innsbruck', 
1994: 'Lillehammer',
1960: 'Squaw Valley',
1976: 'Innsbruck'}

In [13]:
# Dictionary for the host countries key is the year of the games and the country is the value 
countryWDict =  { 1972: 'JPN', 1968: 'FRA', 2002: 'USA', 1980: 'USA', 2018: 'KOR', 
1992: 'FRA', 
1998: 'JPN', 
2006: 'ITA', 
2014: 'RUS', 
2010: 'CAN', 
1984: 'BIH', 
1988: 'CAN', 
1964: 'AUS', 
1994: 'NOR',
1960: 'USA',
1976: 'AUS'}

In [14]:
# Emoty list to hold the dataFrames one for each summer games 
WdataFrameList = []
# Starting with the 1960 olympics 
year = 1960

# For loop of 15 iterations one for every sheet/ games in the xls file 
for x in range(16):
    df = pd.read_excel(xlsW, str(year))

    #First we'll drop all the null columns with no information
    if(year <= 2010):
        df = df.drop(df.columns[[4, 9, 14, 19, 24, 25, 26, 27, 28]], axis=1)
    else:
        df = df.drop(df.columns[[4, 9, 14, 19]], axis=1)
    
    
    # Then we created a small df fro each of the 5 dfs of 4 coulmns seen in the image above 
    j1 = df[['NOC', 'M', 'F', 'Total']].copy()
    j2 = df[['NOC.1', 'M.1', 'F.1', 'Total.1']].copy()
    j3 = df[['NOC.2', 'M.2', 'F.2', 'Total.2']].copy()
    j4 = df[['NOC.3', 'M.3', 'F.3', 'Total.3']].copy()
    j5 = df[['NOC.4', 'M.4', 'F.4', 'Total.4']].copy()
    
    # then we created a list of the names of each column so we could change all the sub dfs column to be the same for joining 
    j2n = list(j2)
    j3n = list(j3)
    j4n = list(j4)
    j5n = list(j5)
    
    # Then we could rename the columns of the 5 smaller dfs
    j2 = j2.rename(columns={ j2n[0]: 'NOC', j2n[1]: 'M', j2n[2]: 'F', j2n[3]: 'Total'})
    j3 = j3.rename(columns={ j3n[0]: 'NOC', j3n[1]: 'M', j3n[2]: 'F', j3n[3]: 'Total'})
    j4 = j4.rename(columns={ j4n[0]: 'NOC', j4n[1]: 'M', j4n[2]: 'F', j4n[3]: 'Total'})
    j5 = j5.rename(columns={ j5n[0]: 'NOC', j5n[1]: 'M', j5n[2]: 'F', j5n[3]: 'Total'})
    
    # Here we are joining the 5 smaller dfs and making one big one
    df = pd.concat([j1, j2, j3, j4, j5], ignore_index=True)
    
    # Dropping all the null values these null rows would be the whitespace rows seen in the image above 
    df = df.dropna(axis=0, how='any')
    
    # Adding a year column with the year of the given games 
    df['Year'] = year
    
    # Adding the host country column referencing its respective dictionary  
    df['Host_Country'] = countryWDict[year]
    
   # Adding the host country column referencing its respective dictionary   
    df['Host_City'] = cityWDict[year]
    
    # Creating a list of dataframes each df in the list represents the numbers from one summer games 
    WdataFrameList.append(df)
    
    # Changing the year to reference the next sheet in the xls file
    if(year == 1992 ):
        year = year + 2
    else:
        year = year + 4

In [15]:
# Now we create the final dataFrame containing all the summer games and the infomration on athletes sent 
windf = pd.concat(WdataFrameList, ignore_index=True)

In [16]:
# creating the summer and winter games bolleans
windf['Summer']= False
windf['Winter']= True

# Final DataFrame 
Now that we have two dataframes for the summer and winter games with fields supplying information on the number of athletes from every country spilt into genders as well as the year of the games, booleans for whether the games were summer or winter and the host country and city of that games.
Now we can combine the two to make our final big dataframe for this section. 

In [17]:
# joining the summer and winter games 
df = pd.concat([sumdf, windf], ignore_index=True)

In [18]:
# Changing these fields to ints from floats 
df['M']= df.M.astype(int)
df['F']= df.F.astype(int)
df['Total']= df.Total.astype(int)

In [19]:
df.to_csv(r"..\..\data\prep\Games\Games-600.csv", index=False)

In [20]:
df

,NOC,M,F,Total,Year,Host_Country,Host_City,Summer,Winter
0,EGY,74,0,74,1960,ITA,Rome,True,False
1,ETH,9,0,9,1960,ITA,Rome,True,False
2,GHA,13,0,13,1960,ITA,Rome,True,False
3,KEN,28,0,28,1960,ITA,Rome,True,False
4,LBR,4,0,4,1960,ITA,Rome,True,False
5,MAR,48,0,48,1960,ITA,Rome,True,False
6,NGR,12,0,12,1960,ITA,Rome,True,False
7,UGA,10,0,10,1960,ITA,Rome,True,False
8,RSA,35,2,37,1960,ITA,Rome,True,False
9,SUD,10,0,10,1960,ITA,Rome,True,False


In [21]:
df[df.NOC == 'OAR']

,NOC,M,F,Total,Year,Host_Country,Host_City,Summer,Winter
